In [ ]:
# import gym
# from gym import spaces
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from operator import itemgetter

In [ ]:
!pip install sb3_contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 7.2 MB/s eta 0:00:00


## Set up variables

In [ ]:
MIG_PROFILE = {
    1: ["7_40"],
    2: ["4_20", "3_20"],
    3: ["4_20", "2_10", "1_10"],
    4: ["4_20", "1_05", "1_05", "1_05"],
    5: ["3_20", "3_20"],
    6: ["3_20", "2_10", "1_10"],
    7: ["3_20", "1_10", "1_05", "1_05"],
    8: ["2_10", "2_10", "3_20"],
    9: ["2_10", "1_05", "1_05", "3_20"],
    10: ["1_05", "1_05", "2_10", "3_20"],
    11: ["1_05", "1_05", "1_05", "1_05", "3_20"],
    12: ["2_10", "2_10", "2_10", "1_10"],
    13: ["2_10", "1_05", "1_05", "2_10", "1_10"],
    14: ["1_05", "1_05", "2_10", "2_10", "1_10"],
    15: ["2_10", "1_10", "1_05", "1_05", "1_05", "1_05"],
    16: ["1_05", "1_05", "2_10", "1_10", "1_05", "1_05"],
    17: ["1_05", "1_05", "1_10", "1_05", "2_10", "1_05"],
    18: ["1_05", "1_05", "1_10", "1_05", "1_05", "2_10"],
    19: ["1_05", "1_05", "1_05", "1_05", "1_05", "1_05", "1_05"]
}

slice_dur_col_match = {
    1: 'g1_duration',
    2: 'g2_duration',
    3: 'g3_duration',
    4: 'g4_duration',
    7: 'g7_duration'
}

mig_util_energy = {
    0: 40,
    1/7: 120,
    2/7: 160,
    3/7: 200,
    4/7: 240,
    5/7: 250,
    0.7142857142857142: 250,
    6/7: 250,
    0.9999999999999999: 250,
    1: 250
}

## Queue generation

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from operator import itemgetter
import torch
from copy import deepcopy
from fractions import Fraction
from fractions import Fraction

# CREATE QUEUE

def create_bert_train(durbase, dur2, dur3, dur4, dur7):
    dur2.append((durbase)/4.1*2.2)
    dur3.append((durbase)/5.8*2.2)
    dur4.append((durbase)/7.1*2.2)
    dur7.append((durbase)/10.5*2.2)

def create_resnet_train(durbase, dur2, dur3, dur4, dur7):
    dur2.append((durbase)/6*3.4)
    dur3.append((durbase)/7.85*3.4)
    dur4.append((durbase)/8.4*3.4)
    dur7.append((durbase)/9.75*3.4)

def create_bert_inf(durbase, dur2, dur3, dur4, dur7):
    dur2.append((durbase)/2)
    dur3.append((durbase)/3)
    dur4.append((durbase)/4)
    dur7.append((durbase)/7)

def create_resnet_inf(durbase, dur2, dur3, dur4, dur7):
    dur2.append((durbase)/2)
    dur3.append((durbase)/3)
    dur4.append((durbase)/12.5*3.2)
    dur7.append((durbase)/18.4*3.2)

def create_mixed_train(durbase, dur2, dur3, dur4, dur7):
    pct = np.random.randint(0,3)
    if pct == 2:
        create_resnet_train(durbase, dur2, dur3, dur4, dur7)
    if pct == 0 or pct == 1:
        create_bert_train(durbase, dur2, dur3, dur4, dur7)

def create_mixed_inf(durbase, dur2, dur3, dur4, dur7):
    pct = np.random.randint(0,3)
    if pct == 2:
        create_resnet_inf(durbase, dur2, dur3, dur4, dur7)
    if pct == 0 or pct == 1:
        create_bert_inf(durbase, dur2, dur3, dur4, dur7)

#type can be linear, sublinear, cap, or mixed
def create_queue(hour_range=24, perf_type="mixed"):#add arrival rate, speed_cap is 0 to not use it or 1 to use it

    global g4_duration, g2_duration, g1_duration, completion_time, start_time, arrivals, deadlines, earliness
    global lateness, selected_slice
    global g7_duration, g3_duration, min_memory

    arrivals = []
    deadlines = []
    min_memory = []
    g7_duration = []
    g4_duration = []
    g3_duration = []
    g2_duration = []
    g1_duration = []
    start_time = []
    completion_time = []
    earliness = []
    lateness = []
    selected_slice = []
    job_arrival = 0
    done = False

    job_arrival = 0
    job_num = 0


    while done == False:
        job_hour = job_arrival / 60
        interarrival_hour = next(x for x, val in enumerate(interarrivals) if val > job_hour) - 1
        interarrival_rate = interarrivals[interarrival_hour]
        #print("interarrival rate is: ", interarrival_rate)
        next_arr_time = random.expovariate(interarrival_rate * 20)
        #print("next arrival time will be in: ", next_arr_time)
        job_arrival += next_arr_time

        if job_arrival / 60 >= hour_range:
            done = True
            #break

        job_num += 1
        arrivals.append(job_arrival)

        job_type = np.random.binomial(1, 0.8)

        if job_type == 0:
            job_g1_duration = np.random.lognormal(mean=(np.log(40) + np.log(60)) / 2, sigma=(np.log(60) - np.log(40)) / (2 * 1.645)) #inference
            create_mixed_train(job_g1_duration, g2_duration, g3_duration, g4_duration, g7_duration)
            #job_min_memory = np.random.uniform(1, 10)

        elif job_type == 1:
            job_g1_duration = np.random.exponential(3) #inference
            create_mixed_inf(job_g1_duration, g2_duration, g3_duration, g4_duration, g7_duration)
            #job_min_memory = np.random.uniform(5, 30)

        g1_duration.append(job_g1_duration)

        #min_memory.append(job_min_memory)


        job_deadline = job_arrival + np.random.uniform(1 * g7_duration[job_num - 1] , 1.5 * g7_duration[job_num - 1])


        deadlines.append(job_deadline)

    min_memory = [x * 0 for x in g1_duration]
    completion_time = [x * 0 for x in g1_duration]
    start_time = [x * 0 for x in g1_duration]
    earliness = [x * 0 for x in g1_duration]
    lateness = [x * 0 for x in g1_duration]
    selected_slice = [x * 0 for x in g1_duration]
    selected_slice_name = [str(x * 0) for x in g1_duration]
    time_remaining = [x * 0 for x in g1_duration]

    job_df = pd.DataFrame(list(zip(arrivals, deadlines, min_memory, selected_slice, selected_slice_name, start_time, completion_time,
                                   earliness, lateness, time_remaining, g1_duration, g2_duration, g3_duration, g4_duration,
                                   g7_duration)),
                         columns = ['arrivals', 'deadlines', 'min_memory', 'selected_slice', 'selected_slice_name','start_time',
                                    'completion_time', 'earliness', 'lateness', 'time_remaining','g1_duration', 'g2_duration',
                                    'g3_duration', 'g4_duration','g7_duration'])


    job_df.insert(0, "id", job_df.index + 1)

    return job_df


interarrivals = {
    0: 0.111,
    1: 0.083,
    2: 0.085,
    3: 0.1,
    4: 0.137,
    5: 0.169,
    6: 0.171,
    7: 0.169,
    8: 0.179,
    9: 0.191,
    10: 0.201,
    11: 0.188,
    12: 0.17,
    13: 0.177,
    14: 0.168,
    15: 0.171,
    16: 0.163,
    17: 0.138,
    18: 0.12,
    19: 0.111,
    20: 0.129,
    21: 0.116,
    22: 0.106,
    23: 0.104,
    24: 0.111
}

# Define model

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from operator import itemgetter

class SchedulingEnv(gym.Env):
    def __init__(self, gpu_config, passing_in_queue, queue):
        self.gpu_config = gpu_config
        self.queue = queue
        self.passing_in_queue = passing_in_queue
        self.gpu_energy_dict = {}
        self.total_tardiness = 0
        self.num_late_jobs = 0
        self.total_energy_consumption = 0

        if self.passing_in_queue:
          self.queue.insert(0, "id", self.queue.index + 1)

        gpu_id = 0
        slices = []
        slice_id = 0

        for gpu in self.gpu_config:
            self.gpu_energy_dict[gpu_id] = 0
            for s in MIG_PROFILE[gpu]:
                slices.append([gpu_id, slice_id, int(s[0]), 0])
                slice_id += 1
            gpu_id += 1

        self.slices = slices
        self.observation_space = spaces.Dict({
                "next_job": spaces.Box(0, np.inf, shape=(4,), dtype=np.float32),
                "ready_queue_dist": spaces.Box(0,1,shape=(40,),dtype=np.float32),
                "slices": spaces.Box(low=np.tile(np.array([0, 0, 1, 0], dtype=np.float32), (len(slices), 1)),
                                     high=np.tile(
                                          np.array([len(self.gpu_config)-1, len(slices) - 1, 7, 1], dtype=np.float32),
                                          (len(slices), 1)
                                     ),
                                     shape=(len(slices), 4),
                                     dtype=np.float32) # gpu_id, slice_id, size, available
            }
        )
        # [time(0-1),[state distribution](12d),[deadline](10d), [avg dur small slice](10d),[avg dur medium slice](10d),[avg dur large slice](10d)]

        self.action_space = spaces.Discrete(len(slices))

        self.np_random = None

    def seed(self, seed=None):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]


    def reset(self, *, seed=None, options=None): #wipe queue, reset to starting migconfig, wipe clock
        super().reset(seed=seed)
        self.np_random, _ = gym.utils.seeding.np_random(seed)

        #reset state
        self.gpu_energy_dict = {}
        self.total_tardiness = 0
        self.num_late_jobs = 0
        self.total_energy_consumption = 0
        # pick random gpu config
        # self.gpu_config = [random.randint(1,19) for _ in range(8)]
        # print(self.gpu_config)


        gpu_id = 0
        slices = []
        slice_id = 0

        for gpu in self.gpu_config:
            self.gpu_energy_dict[gpu_id] = 0
            for s in MIG_PROFILE[gpu]:
                slices.append([gpu_id, slice_id, int(s[0]), 0])
                slice_id += 1
            gpu_id += 1

        self.slices = slices

        if not self.passing_in_queue:
          # create a new queue
          # perf_type = 'mixed'
          # arrival_rate = random.random() * 5 #between 0 and 5
          df = create_queue()
          self.queue = df.copy()
          # self.queue.insert(0, "id", self.queue.index + 1)
        self.working_queue = []

        # initialize event list
        self.event_list = []
        for row in self.queue.itertuples():
            job_id = row.id
            arrival_time = row.arrivals
            job_deadline = row.deadlines
            self.event_list.append((arrival_time, 'arrival', job_id, job_deadline))

        self.working_queue.append(self.event_list[0][2])
        self.now = self.event_list[0][0]
        self.event_list.pop(0)

        return self._get_obs(), {}

    def _get_obs(self):

        working_jobs = self.queue[self.queue['id'].isin(self.working_queue)].copy()

        if working_jobs.empty:
            deadline_dist = [0.0] * 10
            avg_duration_small_dist = [0.0] * 10
            avg_duration_medium_dist = [0.0] * 10
            avg_duration_large_dist = [0.0] * 10
        else:
            rel_deadline = working_jobs['deadlines'] - self.now
            avg_duration_small = (working_jobs['g1_duration'] + working_jobs['g2_duration']) / 2
            avg_duration_medium = (working_jobs['g3_duration'] + working_jobs['g4_duration']) / 2
            avg_duration_large = working_jobs['g7_duration']

            job_bins = [-100, 0, 0.05, 0.2, 0.5, 1, 5, 10, 20, 30, np.inf]

            def proportion_in_bins(series: pd.Series, bins):
                cut = pd.cut(series, bins=bins, right=False, include_lowest=True)
                categories = pd.IntervalIndex.from_breaks(bins, closed='left')
                return (cut.value_counts(normalize=True)
                            .reindex(categories, fill_value=0.0)
                            .to_list())
            deadline_dist = proportion_in_bins(rel_deadline, job_bins)
            avg_duration_small_dist = proportion_in_bins(avg_duration_small, job_bins)
            avg_duration_medium_dist = proportion_in_bins(avg_duration_medium, job_bins)
            avg_duration_large_dist = proportion_in_bins(avg_duration_large, job_bins)

        self.working_queue = sorted(self.working_queue, key=lambda jid: self.queue.loc[self.queue["id"] == jid, "deadlines"].iat[0])

        next_job_chars = np.array([0,0,0,0], dtype=np.float32).flatten()
        if len(self.working_queue) > 0:
            job_row = self.queue.loc[self.queue["id"] == self.working_queue[0]]
            next_job_chars = np.array([job_row['deadlines'] - self.now,
                             (job_row['g1_duration'] + job_row['g2_duration']) / 2,
                             (job_row['g3_duration'] + job_row['g4_duration']) / 2,
                             job_row['g7_duration']], dtype=np.float32).flatten()

        obs = {
                "next_job": next_job_chars,
                "ready_queue_dist": np.array(list(deadline_dist) + list(avg_duration_small_dist)
                         + list(avg_duration_medium_dist) + list(avg_duration_large_dist), dtype=np.float32),
                "slices": np.array(self.slices, dtype=np.float32) # gpu_id, slice_id, size, busy
            }

        return obs

    def calculate_energy(self, gpu_id):
        busy_slices_in_gpu = [s for s in self.slices if s[0] == gpu_id and s[3] == 1]
        total_size = sum([s[2] for s in busy_slices_in_gpu])
        energy = mig_util_energy[total_size/7] * (self.now-self.gpu_energy_dict[gpu_id])
        self.total_energy_consumption += energy

        self.gpu_energy_dict[gpu_id] = self.now
        return energy

    def valid_action_mask(self):
        return np.array([False if s[3]==1 else True for s in self.slices])

    def step(self, action):
        selected_slice = self.slices[action]
        assert(selected_slice[1] == action)

        # if invalid action
        if selected_slice[3] == 1:
          print('invalid action, should not happen?')
          obs = self._get_obs()
          assert(selected_slice[3] == 0)
          # reward = -10 when invalid action
          # return obs, -10, False, False, {}

        self.slices[action][3] = 1
        self.calculate_energy(selected_slice[0])
        job_id = self.working_queue.pop(0)
        job_row = self.queue.loc[self.queue["id"] == job_id]
        dur = job_row[slice_dur_col_match[selected_slice[2]]].iloc[0]

        completion_event = (self.now + dur, 'completion', job_id, selected_slice[0], selected_slice[1], self.now)
        self.event_list.append(completion_event)

        free_slices = [s for s in self.slices if s[3] == 0]
        if len(self.working_queue) > 0 and len(free_slices) > 0:
          obs = self._get_obs()
          mask = self.valid_action_mask()
          # reward = 0 when assigning
          return obs, 0, False, False, {'in_ready_queue': True,
                                        'working_queue_len': len(self.working_queue),
                                        'event_queue_len': len(self.event_list),
                                        'action_mask': mask}

        step_tardiness = 0
        step_energy = 0
        while True:
            self.event_list = sorted(self.event_list, key=lambda x: x[0])
            try:
                event = self.event_list[0]
            except IndexError as e:
                print(self.slices)
                print(self.working_queue)
                print(self.event_list)
                raise e
            event = self.event_list.pop(0)
            self.now = event[0]
            if event[1] == 'arrival':
                self.working_queue.append(event[2])

            elif event[1] == 'completion':
                # Set tardiness
                deadline = self.queue[self.queue['id'] == event[2]]['deadlines'].iloc[0]
                tardiness = max(0, self.now-deadline)
                if tardiness > 0:
                    self.total_tardiness += tardiness
                    self.num_late_jobs += 1
                    step_tardiness += tardiness

                # Update slice to be free
                self.slices[event[4]][3] = 0
                # self.working_queue.remove(event[2])


            # Preemption (occurs when job arrives or finishes)
            completion_events = [e for e in self.event_list if e[1] == 'completion']
            self.event_list = [e for e in self.event_list if e[1] != 'completion']
            for e in completion_events:
                started_at = e[5]
                finish_time = e[0]
                pct_complete = (self.now-started_at)/(finish_time-started_at)

                self.queue.loc[self.queue.loc[self.queue['id'] == e[2]].index[0], "g7_duration"] *= (1-pct_complete)
                self.queue.loc[self.queue.loc[self.queue['id'] == e[2]].index[0], "g4_duration"] *= (1-pct_complete)
                self.queue.loc[self.queue.loc[self.queue['id'] == e[2]].index[0], "g3_duration"] *= (1-pct_complete)
                self.queue.loc[self.queue.loc[self.queue['id'] == e[2]].index[0], "g2_duration"] *= (1-pct_complete)
                self.queue.loc[self.queue.loc[self.queue['id'] == e[2]].index[0], "g1_duration"] *= (1-pct_complete)

                deadline = self.queue[self.queue['id'] == e[2]]['deadlines'].iloc[0]
                if deadline < self.now:
                  step_tardiness += self.now - deadline
                self.working_queue.append(e[2])


            gpu_ids = range(len(self.gpu_config))
            for gpu_id in gpu_ids:
                step_energy += self.calculate_energy(gpu_id)

            # Free all slices
            for s in self.slices:
                s[3] = 0

            if len(self.working_queue) > 0 or len(self.event_list) == 0:
                break

        reward = 0
        if len(completion_events) > 0:
          reward = (-step_tardiness - 0.0000225*step_energy)/(len(completion_events) * (0.0000225+1))

        terminated = False
        if len(self.event_list) == 0 and len(self.working_queue) == 0:
            terminated = True

        info = {'in_ready_queue': False, 'working_queue_len': len(self.working_queue), 'event_queue_len': len(self.event_list), 'total_energy': self.total_energy_consumption,}
        if terminated:
          reward = (-self.total_tardiness - 0.0000225*self.total_energy_consumption)/(len(self.queue) * 0.0000225+1)
          info = {'total_energy': self.total_energy_consumption, 'avg_tardiness': self.total_tardiness/len(self.queue)}

        obs = self._get_obs()
        mask = self.valid_action_mask()
        info['action_mask'] = mask

        truncated = False

        return obs, reward, terminated, truncated, info

# Train model

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, CallbackList
from sb3_contrib.ppo_mask import MaskablePPO
from sb3_contrib.common.maskable.callbacks import MaskableEvalCallback
from sb3_contrib.common.wrappers import ActionMasker

def mask_fn(env):
    return env.valid_action_mask()

def make_masked_env(gpu_config):
    def _init():
        env = SchedulingEnv(gpu_config=gpu_config, queue=None, passing_in_queue=False)
        return ActionMasker(env, mask_fn)
    return _init

# Vectorized training env
train_env = DummyVecEnv([make_masked_env([1, 1, 2, 2, 3, 3, 12, 12]) for _ in range(16)])

# Vectorized evaluation env
eval_env = DummyVecEnv([make_masked_env([1, 1, 2, 2, 3, 3, 12, 12]) for _ in range(16)])

# Callbacks
eval_callback = MaskableEvalCallback(
    eval_env,
    best_model_save_path='./best_model/',
    log_path='./logs/',
    eval_freq=5000
)

checkpoint_callback = CheckpointCallback(
    save_freq=10000,
    save_path='./checkpoints/',
    name_prefix="ppo_sched"
)

callback = CallbackList([eval_callback, checkpoint_callback])

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import os

model = MaskablePPO(
    "MultiInputPolicy",
    train_env,
    verbose=1,
    tensorboard_log=None,
    n_steps=1024,
    batch_size=2048,
    n_epochs=5,
    learning_rate=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.001,
    vf_coef=0.5,
    max_grad_norm=0.5,
    # tensorboard_log="./ppo_sched_log",
    policy_kwargs=dict(net_arch=[256, 256]),
)

model.learn(total_timesteps=200_000, callback=callback)

model_dir = os.environ.get("AIP_MODEL_DIR", "./model_new_queue")
model.save(os.path.join(model_dir, "ppo_sched_model"))

Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

------------------------------
| time/              |       |
|    fps             | 37    |
|    iterations      | 1     |
|    time_elapsed    | 437   |
|    total_timesteps | 16384 |
------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 2           |
|    time_elapsed         | 880         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008888401 |
|    clip_fraction        | 0.0572      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.69       |
|    explained_variance   | -0.254      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.41        |
|    n_updates            | 5           |
|    policy_gradient_loss | -0.00328    |
|    value_loss           | 1.01        |
-----------------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


------------------------------------------
| time/                   |              |
|    fps                  | 37           |
|    iterations           | 3            |
|    time_elapsed         | 1318         |
|    total_timesteps      | 49152        |
| train/                  |              |
|    approx_kl            | 0.0069553284 |
|    clip_fraction        | 0.0146       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.55        |
|    explained_variance   | -0.00338     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.57         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00149     |
|    value_loss           | 1.49         |
------------------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


------------------------------------------
| time/                   |              |
|    fps                  | 37           |
|    iterations           | 4            |
|    time_elapsed         | 1756         |
|    total_timesteps      | 65536        |
| train/                  |              |
|    approx_kl            | 0.0065750484 |
|    clip_fraction        | 0.017        |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.63        |
|    explained_variance   | 0.00718      |
|    learning_rate        | 0.0003       |
|    loss                 | 1.43         |
|    n_updates            | 15           |
|    policy_gradient_loss | -0.000819    |
|    value_loss           | 3.29         |
------------------------------------------


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replac

# Evaluate model

In [ ]:
queues = [create_queue() for _ in range(5)]

In [ ]:
from sb3_contrib.common.maskable.utils import get_action_masks

model = MaskablePPO.load("./model_new_queue/ppo_sched_model.zip")

total_energy_consumption = []
avg_tardiness = []

for i in range(5):
    print(i)
    queue = queues[i].drop('id', axis=1)
    env = ActionMasker(SchedulingEnv([1, 1, 2, 2, 3, 3, 12, 12], True, queue), mask_fn)
    obs, _ = env.reset()
    done = False
    while done == False:
        action_masks = get_action_masks(env)
        action, _states = model.predict(obs, action_masks=action_masks)
        obs, reward, terminated, truncated, info = env.step(action)
        # print(info)
        done = terminated or truncated
        if done:
            print(info)
            total_energy_consumption.append(info['total_energy'])
            avg_tardiness.append(info['avg_tardiness'])

print(total_energy_consumption)
print(avg_tardiness)